In [2]:
import numpy as np
import tensorflow as tf
from glob import glob
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras import backend as K
from PIL import Image

2024-12-12 17:05:46.567830: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-12 17:05:46.608793: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-12 17:05:46.917083: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-12 17:05:46.919044: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-12 17:05:47.896966: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
def resize(path, width, height):
    img = Image.open(path)
    new_image = img.resize((width, height))
    new_image = np.array(new_image)
    if new_image.shape[2] > 3:
        print(f"No rgb image detected: {path}")
    return new_image

In [4]:
paths = glob("images/*")

In [5]:
height = 500
width = 500
x_train = []
for path in paths:
    x_train.append(resize(path, height, width))

In [6]:
# for image in x_train:
#     print(image.shape)

In [7]:
x_train = np.array(x_train)

In [8]:
n_samples, _, _, n_bands = x_train.shape

In [9]:
x_train = x_train.reshape(n_samples, height*width*n_bands)

In [11]:
# Hyperparametersx_t
latent_dim = 3  # Dimensionality of the latent space
input_dim = height * width * n_bands  # Input dimension (e.g., flattened MNIST images)

In [12]:
# Encoder
inputs = Input(shape=(input_dim,), name="encoder_input")
h = Dense(2048, activation='relu')(inputs)
h = Dense(512, activation='relu')(h)
h = Dense(128, activation='relu')(h)

z_mean = Dense(latent_dim, name="z_mean")(h)
z_log_var = Dense(latent_dim, name="z_log_var")(h)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,), name="z")([z_mean, z_log_var])

# Decoder
latent_inputs = Input(shape=(latent_dim,), name="z_sampling")
h_decoded = Dense(128, activation='relu')(latent_inputs)
h_decoded = Dense(512, activation='relu')(h_decoded)
h_decoded = Dense(2048, activation='relu')(h_decoded)
outputs = Dense(input_dim, activation='sigmoid')(h_decoded)

# Models
encoder = Model(inputs, [z_mean, z_log_var, z], name="encoder")
decoder = Model(latent_inputs, outputs, name="decoder")
vae_outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, vae_outputs, name="vae")

2024-12-12 17:06:24.040660: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (mklcpu) ran out of memory trying to allocate 5.72GiB (rounded to 6144000000)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-12-12 17:06:24.041197: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for mklcpu
2024-12-12 17:06:24.041221: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-12-12 17:06:24.041233: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-12-12 17:06:24.041244: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (1024): 	Total 

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[750000,2048] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:Mul] name: 

In [ ]:
# Loss function
reconstruction_loss = MeanSquaredError()(inputs, vae_outputs) * input_dim
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [ ]:
# Train the VAE
vae.fit(x_train, epochs=10, batch_size=32)

In [ ]:
# Sampling from the latent space
def generate_samples(decoder, n_samples=5):
    z_samples = np.random.normal(size=(n_samples, latent_dim))
    generated = decoder.predict(z_samples)
    return generated

In [ ]:
# Generate new data
new_data = generate_samples(decoder, n_samples=10)
print("Generated Data:", new_data)